#　チェックポイント

In [ ]:
from github import Github
from github import Auth
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
auth = Auth.Token()
g = Github(auth=auth)

In [65]:
import re

def find_all_matches(pattern, string, group=0):
    pos = 0
    out = []
    while m := pattern.search(string, pos):
        pos = m.start() + 1
        out.append(m[group])
    return out

def diff_header_parser(diff: str) -> list[int]:
    # print(diff)
    regex = re.compile(r"(@@) ([\-\+][0-9]+,[0-9]+ )+(@@)")

    m = find_all_matches(regex, diff)
    
    if m == None:
        return [0, 0]

    # print(m)

    minus = 0
    plus = 0

    for l in m:
        group: list[str] = re.findall(r"[\-\+][1-9]+,[1-9]+", l)

        for x in group:
            d = re.search(r"([\-\+])([0-9]+),([0-9]+)", x)
            # print(d.groups())
            if d.groups()[0] == '-':
                minus += int(d.groups()[2])
            else:
                plus += int(d.groups()[2])



    return [minus, plus]

In [ ]:
# spring_pr_df['diff_url'].to_dict()[0]

# import requests
# r = requests.get(spring_pr_df['diff_url'].to_dict()[0])
# r.raise_for_status()
# diff = r.text

# diff_header_parser(diff)

[30, 69]

In [ ]:


# for x in spring_pr_df['diff_url'].tolist():
#     r = requests.get(x)
#     r.raise_for_status()
#     diff = r.text
#     changed_lines = diff_header_parser(diff=diff)
#     minus_lines.append(changed_lines[0])
#     plus_lines.append(changed_lines[1])



In [69]:
from github.PaginatedList import PaginatedList
from github.Issue import Issue
from github.PullRequest import PullRequest
import requests

def convert_prs_to_dataframe(issues: PaginatedList[Issue]): 
    pr_list = [d.as_pull_request() for d in issues]
    minus_lines = []
    plus_lines = []
    for x in pr_list:
        r = requests.get(x.diff_url)
        r.raise_for_status()
        changed_lines = diff_header_parser(diff=r.text)
        minus_lines.append(changed_lines[0])
        plus_lines.append(changed_lines[1])
    data = dict.fromkeys(pr_list[0].raw_data.keys())
    for key in data.keys():
        data[key] = [d.raw_data[key] for d in pr_list]
    data['last_modified'] = [d.last_modified for d in pr_list]
    data['last_modified_datetime'] = [d.last_modified_datetime for d in pr_list]
    data['minus_lines'] = minus_lines
    data['plus_lines'] = plus_lines
    return pd.DataFrame(data)

In [70]:
spring_prs = g.search_issues(query='repo:spring-projects/spring-framework created:>2025-02-15 is:pr')
spring_pr_df = convert_prs_to_dataframe(spring_prs)
spring_pr_df

,url,id,node_id,html_url,diff_url,patch_url,issue_url,number,state,locked,...,review_comments,maintainer_can_modify,commits,additions,deletions,changed_files,last_modified,last_modified_datetime,minus_lines,plus_lines
0,https://api.github.com/repos/spring-projects/s...,2433009173,PR_kwDOABGHUc6RBMYV,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34701,open,False,...,0,True,1,43,0,2,"Wed, 02 Apr 2025 06:44:08 GMT",2025-04-02 06:44:08+00:00,30,69
1,https://api.github.com/repos/spring-projects/s...,2430485627,PR_kwDOABGHUc6Q3kR7,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34697,open,False,...,1,True,1,57,5,2,"Wed, 02 Apr 2025 06:07:27 GMT",2025-04-02 06:07:27+00:00,7,24
2,https://api.github.com/repos/spring-projects/s...,2430435774,PR_kwDOABGHUc6Q3YG-,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34696,open,False,...,0,True,1,13,3,1,"Tue, 01 Apr 2025 04:04:00 GMT",2025-04-01 04:04:00+00:00,25,44
3,https://api.github.com/repos/spring-projects/s...,2427495516,PR_kwDOABGHUc6QsKRc,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34686,closed,False,...,0,False,1,1,1,1,"Mon, 31 Mar 2025 12:29:36 GMT",2025-03-31 12:29:36+00:00,7,7
4,https://api.github.com/repos/spring-projects/s...,2426970185,PR_kwDOABGHUc6QqKBJ,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34683,open,False,...,0,True,1,219,3,7,"Sat, 29 Mar 2025 15:08:59 GMT",2025-03-29 15:08:59+00:00,158,277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,https://api.github.com/repos/spring-projects/s...,2343496482,PR_kwDOABGHUc6Lrusi,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34447,closed,False,...,0,False,1,12,6,1,"Mon, 10 Mar 2025 11:06:40 GMT",2025-03-10 11:06:40+00:00,21,27
62,https://api.github.com/repos/spring-projects/s...,2342298859,PR_kwDOABGHUc6LnKTr,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34445,closed,False,...,0,False,1,1,1,1,"Thu, 27 Mar 2025 03:04:17 GMT",2025-03-27 03:04:17+00:00,7,7
63,https://api.github.com/repos/spring-projects/s...,2341591071,PR_kwDOABGHUc6Lkdgf,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34444,closed,False,...,0,False,1,54,1,2,"Sun, 30 Mar 2025 18:51:10 GMT",2025-03-30 18:51:10+00:00,7,60
64,https://api.github.com/repos/spring-projects/s...,2340452635,PR_kwDOABGHUc6LgHkb,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34440,closed,False,...,0,False,1,1,1,1,"Sat, 22 Mar 2025 14:00:27 GMT",2025-03-22 14:00:27+00:00,7,7


In [41]:
spring_pr_df['diff_url'].to_dict()

{0: 'https://github.com/spring-projects/spring-framework/pull/34701.diff',
 1: 'https://github.com/spring-projects/spring-framework/pull/34697.diff',
 2: 'https://github.com/spring-projects/spring-framework/pull/34696.diff',
 3: 'https://github.com/spring-projects/spring-framework/pull/34686.diff',
 4: 'https://github.com/spring-projects/spring-framework/pull/34683.diff',
 5: 'https://github.com/spring-projects/spring-framework/pull/34682.diff',
 6: 'https://github.com/spring-projects/spring-framework/pull/34681.diff',
 7: 'https://github.com/spring-projects/spring-framework/pull/34679.diff',
 8: 'https://github.com/spring-projects/spring-framework/pull/34673.diff',
 9: 'https://github.com/spring-projects/spring-framework/pull/34669.diff',
 10: 'https://github.com/spring-projects/spring-framework/pull/34668.diff',
 11: 'https://github.com/spring-projects/spring-framework/pull/34667.diff',
 12: 'https://github.com/spring-projects/spring-framework/pull/34665.diff',
 13: 'https://github.c

In [62]:
spring_pr_df['diff_url'].tolist()

['https://github.com/spring-projects/spring-framework/pull/34701.diff',
 'https://github.com/spring-projects/spring-framework/pull/34697.diff',
 'https://github.com/spring-projects/spring-framework/pull/34696.diff',
 'https://github.com/spring-projects/spring-framework/pull/34686.diff',
 'https://github.com/spring-projects/spring-framework/pull/34683.diff',
 'https://github.com/spring-projects/spring-framework/pull/34682.diff',
 'https://github.com/spring-projects/spring-framework/pull/34681.diff',
 'https://github.com/spring-projects/spring-framework/pull/34679.diff',
 'https://github.com/spring-projects/spring-framework/pull/34673.diff',
 'https://github.com/spring-projects/spring-framework/pull/34669.diff',
 'https://github.com/spring-projects/spring-framework/pull/34668.diff',
 'https://github.com/spring-projects/spring-framework/pull/34667.diff',
 'https://github.com/spring-projects/spring-framework/pull/34665.diff',
 'https://github.com/spring-projects/spring-framework/pull/34657

In [72]:
rejected = spring_pr_df[(spring_pr_df['state'] == 'closed') & (spring_pr_df['merged'] == False)]
accepted = spring_pr_df[(spring_pr_df['state'] == 'closed') & (spring_pr_df['merged'] == True)]

In [85]:
rejected['additions'].tolist()
accepted['additions'].tolist()

[1, 22, 150, 9, 37]

In [36]:
from scipy.stats import mannwhitneyu

mannwhitneyu(rejected['additions'].tolist(), accepted['additions'].tolist())

MannwhitneyuResult(statistic=np.float64(126.0), pvalue=np.float64(0.8669594640261339))

In [37]:
mannwhitneyu(rejected['changed_files'].tolist(), accepted['changed_files'].tolist())

MannwhitneyuResult(statistic=np.float64(113.0), pvalue=np.float64(0.8359433984489774))

In [38]:
mannwhitneyu(rejected['deletions'].tolist(), accepted['deletions'].tolist())

MannwhitneyuResult(statistic=np.float64(110.5), pvalue=np.float64(0.782569427856017))

In [ ]:
mannwhitneyu(rejected['review_comments'].tolist(), accepted['review_comments'].tolist())

MannwhitneyuResult(statistic=np.float64(118.0), pvalue=np.float64(0.9443097405113265))

In [74]:
mannwhitneyu(rejected['minus_lines'].tolist(), accepted['minus_lines'].tolist())

MannwhitneyuResult(statistic=np.float64(103.5), pvalue=np.float64(0.6259782016033182))

In [75]:
mannwhitneyu(rejected['plus_lines'].tolist(), accepted['plus_lines'].tolist())

MannwhitneyuResult(statistic=np.float64(106.5), pvalue=np.float64(0.6921222098964378))

In [76]:
closed = spring_pr_df[(spring_pr_df['state'] == 'closed')]
closed

,url,id,node_id,html_url,diff_url,patch_url,issue_url,number,state,locked,...,review_comments,maintainer_can_modify,commits,additions,deletions,changed_files,last_modified,last_modified_datetime,minus_lines,plus_lines
3,https://api.github.com/repos/spring-projects/s...,2427495516,PR_kwDOABGHUc6QsKRc,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34686,closed,False,...,0,False,1,1,1,1,"Mon, 31 Mar 2025 12:29:36 GMT",2025-03-31 12:29:36+00:00,7,7
5,https://api.github.com/repos/spring-projects/s...,2426935073,PR_kwDOABGHUc6QqBch,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34682,closed,False,...,0,False,1,33,5,2,"Tue, 01 Apr 2025 07:57:00 GMT",2025-04-01 07:57:00+00:00,41,51
7,https://api.github.com/repos/spring-projects/s...,2426787351,PR_kwDOABGHUc6QpdYX,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34679,closed,False,...,1,False,1,13,13,10,"Sat, 29 Mar 2025 13:10:51 GMT",2025-03-29 13:10:51+00:00,63,63
9,https://api.github.com/repos/spring-projects/s...,2424876720,PR_kwDOABGHUc6QiK6w,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34669,closed,False,...,0,False,1,51,1,2,"Fri, 28 Mar 2025 13:37:14 GMT",2025-03-28 13:37:14+00:00,27,50
10,https://api.github.com/repos/spring-projects/s...,2424653053,PR_kwDOABGHUc6QhUT9,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34668,closed,False,...,0,False,1,22,28,4,"Fri, 28 Mar 2025 13:40:52 GMT",2025-03-28 13:40:52+00:00,113,125
11,https://api.github.com/repos/spring-projects/s...,2424634047,PR_kwDOABGHUc6QhPq_,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34667,closed,False,...,0,False,1,188,99,6,"Fri, 28 Mar 2025 13:56:05 GMT",2025-03-28 13:56:05+00:00,249,325
13,https://api.github.com/repos/spring-projects/s...,2419660682,PR_kwDOABGHUc6QOReK,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34657,closed,False,...,0,False,1,4,0,2,"Wed, 26 Mar 2025 21:42:31 GMT",2025-03-26 21:42:31+00:00,12,16
14,https://api.github.com/repos/spring-projects/s...,2419057833,PR_kwDOABGHUc6QL-Sp,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34656,closed,False,...,0,False,1,312,275,9,"Wed, 26 Mar 2025 17:58:17 GMT",2025-03-26 17:58:17+00:00,355,387
15,https://api.github.com/repos/spring-projects/s...,2417521749,PR_kwDOABGHUc6QGHRV,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s...,34655,closed,False,...,0,False,1,106,15,2,"Wed, 02 Apr 2025 08:23:35 GMT",2025-04-02 08:23:35+00:00,73,164
17,https://api.github.com/repos/spring-projects/s...,2411336957,PR_kwDOABGHUc6PuhT9,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://github.com/spring-projects/spring-fram...,https://api.github.com/repos/spring-projects/s.

In [78]:
X = closed[['minus_lines', 'plus_lines']]
Y = closed.merged

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=16)

logreg = LogisticRegression(random_state=16)
logreg.fit(X_train, y_train)



LogisticRegression(random_state=16)

In [84]:
Y

3      True
5     False
7     False
9     False
10     True
11    False
13    False
14    False
15    False
17    False
18     True
19    False
21     True
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
33    False
34    False
35    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
59     True
60    False
61    False
62    False
63    False
64    False
65    False
Name: merged, dtype: bool

In [79]:
y_pred = logreg.predict(X_test)

In [80]:
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

/home/panxiao81/naist-2025-internship/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


array([[11]])